In [ ]:
import numpy as np

np.set_printoptions(threshold=5)

# Unit-Aware Calculations
Where possible, HuracanPy functions are doing unit-aware calculations.
This is achieved using functionality from [metpy and the metpy accessor](https://unidata.github.io/MetPy/latest/tutorials/xarray_tutorial.html#units).

Typically, for metpy unit-aware calculations to work, the units must be included in the attributes of the variable. e.g.

In [ ]:
import huracanpy

tracks = huracanpy.load(huracanpy.example_csv_file)

# Add units to some variables
tracks.slp.attrs["units"] = "Pa"
tracks.wind10.attrs["units"] = "m s-1"
tracks.lon.attrs["units"] = "degrees"
tracks.lat.attrs["units"] = "degrees"

# Use a unit-aware huracanpy calculation
dv_dt = tracks.hrcn.get_rate(var_name="wind10")
print(dv_dt)

## MetPy functionality

When the track data has units in the attributes you can use functionalities from the `metpy` accessor
(see [the metpy docs](https://unidata.github.io/MetPy/latest/tutorials/xarray_tutorial.html#units) for more details).

### Unit Conversion
For example, if we want to convert the intensification to more readable units we can
use the metpy unit conversion

In [ ]:
print(dv_dt.metpy.convert_units("m s-1 day-1"))

### dequantify and quantify

Note that, the returned `Dataarray` contains a `pint.Quantity` instead of a numpy array.
This is a numpy-like array that enables the unit-aware calculations. However it can
sometimes result in strange behaviour. The `metpy` accessor provides a method for
undoing this (and HuracanPy functions will do this automatically)

In [ ]:
print(dv_dt.metpy.convert_units("m s-1 day-1").metpy.dequantify())

Using `dequantify` has reverted to a numpy array with the units in the attributes

Be aware that fairly trivial operations will lose these units if they are not done
in a unit-aware way

In [ ]:
print(dv_dt + 0)

If you want to do your own unit-aware calculations, you will need to use the `metpy` methods, e.g.

In [ ]:
from metpy.units import units

print((dv_dt.metpy.quantify() + 0 * units("m s-2")).metpy.dequantify())

## HuracanPy functionality
The following functions support unit-aware calculations

- [huracanpy.calc.delta](../api/_autosummary/huracanpy.calc.delta.rst)
- [huracanpy.calc.rate](../api/_autosummary/huracanpy.calc.rate.rst)
- [huracanpy.calc.distance](../api/_autosummary/huracanpy.calc.distance.rst)
- [huracanpy.calc.translation_speed](../api/_autosummary/huracanpy.calc.translation_speed.rst)
- [huracanpy.info.category](../api/_autosummary/huracanpy.info.category.rst)
- [huracanpy.tc.ace](../api/_autosummary/huracanpy.tc.ace.rst)
- [huracanpy.tc.pace](../api/_autosummary/huracanpy.tc.pace.rst)
- [huracanpy.tc.beta_drift](../api/_autosummary/huracanpy.tc.beta_drift.rst)

### Assumed units
Unless you have a netCDF file which includes units, or manually add the units, your track variables will not have units.
To account for this, HuracanPy functions that use units will assume that your variables have default units with the following convention

- Distance in metres
- Speed in metres per second
- Pressure in hectoPascals (with checks on reasonable values)
- Latitude/longitude in degrees

e.g. calculating accumulated cyclone energy (ACE), the input is assumed to be in metres per second and converted to knots


In [ ]:
# Reload the tracks without units
tracks = huracanpy.load(huracanpy.example_csv_file)

# Calculate ACE without specifying units
ace = tracks.hrcn.get_ace(wind_name="wind10", sum_by="track_id")

# Result has units of knots**2
print(ace)

The units can also be specified directly as a function argument

In [ ]:
# Tell the function that the input wind is in metres per second (same result)
ace = tracks.hrcn.get_ace(wind_name="wind10", sum_by="track_id", wind_units="m s-1")
print(ace)

In [ ]:
# Tell the function that the input wind is in kilometres per second for a wildly
# inaccurate result
ace = tracks.hrcn.get_ace(wind_name="wind10", sum_by="track_id", wind_units="km s-1")
print(ace)

In some cases, the default will check that the values have a reasonable magnitude for
the assumed units. e.g. calculated tropical-cyclone category from pressure the units
of the track are in Pascals but assumed to be hectoPascals if unspecified.

In [ ]:
category = tracks.hrcn.get_pressure_category()
print(category)

If you want to override this check, you can specify the units, either through the attributes or the function arguments

In [ ]:
category = tracks.hrcn.get_pressure_category(slp_units="hPa")
print(category)